# Step 1: Reference Data Analysis

**Load and analyze street network data from GeoJSON files**

**Cities** (500×500m windows):
- London, UK
- Berlin, Germany
- Belgrade, Serbia
- Torino, Italy

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Patch
from pathlib import Path
from collections import Counter
import math
import pickle
import geopandas as gpd
from shapely.geometry import Point, LineString

%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

print("✓ Libraries loaded")

In [ ]:
# Create output directories
from pathlib import Path
Path("outputs/visualizations").mkdir(parents=True, exist_ok=True)
Path("outputs/data").mkdir(parents=True, exist_ok=True)
print("✓ Output directories created")

## Configuration

In [ ]:
CITIES = {
    'london': {
        'name': 'London, UK',
        'coords': (51.5108708294874, -0.1301202729436442),
        'color': '#E74C3C'
    },
    'berlin': {
        'name': 'Berlin, Germany',
        'coords': (52.52832783083204, 13.40299924970717),
        'color': '#3498DB'
    },
    'belgrade': {
        'name': 'Belgrade, Serbia',
        'coords': (44.81648489551224, 20.462214816208164),
        'color': '#2ECC71'
    },
    'torino': {
        'name': 'Torino, Italy',
        'coords': (45.06940684010285, 7.682084193995683),
        'color': '#F39C12'
    }
}

WINDOW_SIZE_M = 500  # 500×500m window
MIN_SEGMENT_LENGTH = 5.0  # Filter segments < 5m

print(f"Window size: {WINDOW_SIZE_M}m × {WINDOW_SIZE_M}m")
print(f"Min segment length: {MIN_SEGMENT_LENGTH}m")

def load_network_from_geojson(city_key, data_dir='inv_city/outputs/geojson'):
    """
    Load street network from existing GeoJSON files.
    Filter for PEDESTRIAN paths only.
    """
    print(f"  Loading from GeoJSON files...")
    
    nodes_path = Path(data_dir) / f"{city_key}_nodes.geojson"
    edges_path = Path(data_dir) / f"{city_key}_edges.geojson"
    
    nodes_gdf = gpd.read_file(nodes_path)
    edges_gdf = gpd.read_file(edges_path)
    
    print(f"  Loaded: {len(nodes_gdf)} nodes, {len(edges_gdf)} edges")
    
    # Filter for PEDESTRIAN ONLY paths
    pedestrian_types = ['footway', 'path', 'pedestrian', 'steps']
    
    def is_pedestrian(highway_val):
        if highway_val is None:
            return False
        if hasattr(highway_val, '__iter__') and not isinstance(highway_val, str):
            highway_val = highway_val[0] if len(highway_val) > 0 else None
        return highway_val in pedestrian_types
    
    edges_gdf = edges_gdf[edges_gdf['highway'].apply(is_pedestrian)]
    print(f"  After pedestrian filter: {len(edges_gdf)} edges")
    
    G = nx.MultiDiGraph()
    
    for idx, row in nodes_gdf.iterrows():
        node_id = row['osmid']
        coords = row.geometry.coords[0]
        G.add_node(node_id, x=coords[0], y=coords[1])
    
    for idx, row in edges_gdf.iterrows():
        u = row['u']
        v = row['v']
        
        if u in G.nodes() and v in G.nodes():
            length = row.geometry.length
            G.add_edge(u, v, length=length, geometry=row.geometry)
    
    print(f"  Graph created: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    
    return G


def clean_and_filter_graph(G, min_length=5.0):
    pos = {node: (data['x'], data['y']) for node, data in G.nodes(data=True)}
    
    edges_to_remove = []
    for u, v, key, data in G.edges(keys=True, data=True):
        length = data.get('length', 0)
        if length < min_length:
            edges_to_remove.append((u, v, key))
    
    G.remove_edges_from(edges_to_remove)
    print(f"  Removed {len(edges_to_remove)} edges < {min_length}m")
    
    isolated = list(nx.isolates(G))
    G.remove_nodes_from(isolated)
    for node in isolated:
        if node in pos:
            del pos[node]
    
    print(f"  Final: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    
    return G, pos


def normalize_to_window(pos, window_size=500):
    if not pos:
        return {}
    
    coords = np.array(list(pos.values()))
    min_x, min_y = coords.min(axis=0)
    max_x, max_y = coords.max(axis=0)
    
    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    
    pos_normalized = {}
    for node, (x, y) in pos.items():
        nx = x - center_x
        ny = y - center_y
        nx += window_size / 2
        ny += window_size / 2
        pos_normalized[node] = (nx, ny)
    
    return pos_normalized


def transform_geometry(geom, offset_x, offset_y):
    from shapely.affinity import translate
    return translate(geom, xoff=offset_x, yoff=offset_y)


print("✓ Helper functions defined")

In [ ]:
def load_network_from_geojson(city_key, data_dir='inv_city/outputs/geojson'):
    """
    Load street network from existing GeoJSON files.
    Filter for PEDESTRIAN paths only.
    
    Args:
        city_key: City identifier
        data_dir: Directory containing GeoJSON files
    
    Returns:
        NetworkX graph (already in projected coordinates)
    """
    print(f"  Loading from GeoJSON files...")
    
    # Load nodes and edges
    nodes_path = Path(data_dir) / f"{city_key}_nodes.geojson"
    edges_path = Path(data_dir) / f"{city_key}_edges.geojson"
    
    nodes_gdf = gpd.read_file(nodes_path)
    edges_gdf = gpd.read_file(edges_path)
    
    print(f"  Loaded: {len(nodes_gdf)} nodes, {len(edges_gdf)} edges")
    
    # Filter for PEDESTRIAN ONLY paths
    pedestrian_types = ['footway', 'path', 'pedestrian', 'steps']
    
    def is_pedestrian(highway_val):
        if highway_val is None:
            return False
        # Handle numpy array
        if hasattr(highway_val, '__iter__') and not isinstance(highway_val, str):
            highway_val = highway_val[0] if len(highway_val) > 0 else None
        return highway_val in pedestrian_types
    
    # Filter edges
    edges_gdf = edges_gdf[edges_gdf['highway'].apply(is_pedestrian)]
    print(f"  After pedestrian filter: {len(edges_gdf)} edges")
    
    # Create NetworkX graph
    G = nx.MultiDiGraph()
    
    # Add nodes with coordinates (already in projected meters)
    for idx, row in nodes_gdf.iterrows():
        node_id = row['osmid']
        coords = row.geometry.coords[0]
        G.add_node(node_id, x=coords[0], y=coords[1])
    
    # Add edges with full geometry
    for idx, row in edges_gdf.iterrows():
        u = row['u']
        v = row['v']
        
        if u in G.nodes() and v in G.nodes():
            length = row.geometry.length
            G.add_edge(u, v, length=length, geometry=row.geometry)
    
    print(f"  Graph created: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    
    return G


def clean_and_filter_graph(G, min_length=5.0):
    """
    Filter short edges and clean graph.
    
    Args:
        G: NetworkX graph (projected)
        min_length: Minimum edge length in meters
    
    Returns:
        Cleaned graph, node positions dict
    """
    # Get node positions
    pos = {node: (data['x'], data['y']) for node, data in G.nodes(data=True)}
    
    # Filter edges by length
    edges_to_remove = []
    for u, v, key, data in G.edges(keys=True, data=True):
        length = data.get('length', 0)
        if length < min_length:
            edges_to_remove.append((u, v, key))
    
    G.remove_edges_from(edges_to_remove)
    print(f"  Removed {len(edges_to_remove)} edges < {min_length}m")
    
    # Remove isolated nodes
    isolated = list(nx.isolates(G))
    G.remove_nodes_from(isolated)
    for node in isolated:
        if node in pos:
            del pos[node]
    
    print(f"  Final: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    
    return G, pos


def normalize_to_window(pos, window_size=500):
    """
    Normalize coordinates to [0, window_size] box.
    
    Args:
        pos: Dict of {node: (x, y)}
        window_size: Target window size
    
    Returns:
        Normalized positions dict
    """
    if not pos:
        return {}
    
    coords = np.array(list(pos.values()))
    min_x, min_y = coords.min(axis=0)
    max_x, max_y = coords.max(axis=0)
    
    # Center and scale
    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    
    pos_normalized = {}
    for node, (x, y) in pos.items():
        # Center at origin
        nx = x - center_x
        ny = y - center_y
        # Shift to positive quadrant
        nx += window_size / 2
        ny += window_size / 2
        pos_normalized[node] = (nx, ny)
    
    return pos_normalized


def transform_geometry(geom, offset_x, offset_y):
    """Transform LineString geometry to normalized window coordinates."""
    from shapely.geometry import LineString
    from shapely.affinity import translate
    
    return translate(geom, xoff=offset_x, yoff=offset_y)


print("✓ Helper functions defined")

def calculate_bearing(p1, p2):
    """Calculate bearing (0-180°)."""
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    angle = math.atan2(dy, dx)
    bearing = math.degrees(angle)
    bearing = bearing % 180
    return bearing


def compute_morphology_metrics(G, pos):
    """Compute morphology metrics from FILTERED graph."""
    metrics = {}

    area_km2 = (WINDOW_SIZE_M / 1000.0) ** 2
    metrics['node_density'] = G.number_of_nodes() / area_km2

    degrees = [d for _, d in G.degree()]
    metrics['degree_distribution'] = dict(Counter(degrees))
    metrics['avg_degree'] = np.mean(degrees) if degrees else 0

    dead_ends = sum(1 for d in degrees if d == 1)
    metrics['dead_end_ratio'] = dead_ends / len(degrees) if degrees else 0

    lengths = []
    for u, v, key, data in G.edges(keys=True, data=True):
        length = data.get('length', 0)
        lengths.append(length)

    metrics['segment_lengths'] = lengths
    metrics['avg_segment_length'] = np.mean(lengths) if lengths else 0

    bearings = []
    for u, v in G.edges():
        bearing = calculate_bearing(pos[u], pos[v])
        bearings.append(bearing)

    if bearings:
        counts, bins = np.histogram(bearings, bins=18, range=(0, 180))
        metrics['orientation_hist'] = (bins, counts)
    else:
        metrics['orientation_hist'] = (np.linspace(0, 180, 19), np.zeros(18))

    return metrics


def compute_building_metrics(buildings_gdf, pedestrian_edges_gdf):
    """Compute building-related metrics."""
    from shapely.ops import unary_union

    metrics = {}
    areas = []
    compactness_values = []

    for _, building in buildings_gdf.iterrows():
        if building.geometry is not None and building.geometry.geom_type == 'Polygon':
            area = building.geometry.area
            perimeter = building.geometry.length
            areas.append(area)

            if area > 0:
                compactness = (perimeter ** 2) / area
            else:
                compactness = 0
            compactness_values.append(compactness)

    metrics['footprint_areas'] = areas
    metrics['avg_footprint_area'] = np.mean(areas) if areas else 0
    metrics['median_footprint_area'] = np.median(areas) if areas else 0

    total_building_area = sum(areas)
    window_area = WINDOW_SIZE_M ** 2
    metrics['building_coverage_ratio'] = total_building_area / window_area
    metrics['building_density'] = len(buildings_gdf) / (window_area / 10000)

    metrics['compactness_values'] = compactness_values
    metrics['avg_compactness'] = np.mean(compactness_values) if compactness_values else 0
    metrics['median_compactness'] = np.median(compactness_values) if compactness_values else 0

    proximities = []
    if len(pedestrian_edges_gdf) > 0:
        all_paths = unary_union(pedestrian_edges_gdf.geometry)

        for _, building in buildings_gdf.iterrows():
            if building.geometry is not None and building.geometry.geom_type == 'Polygon':
                centroid = building.geometry.centroid
                distance = centroid.distance(all_paths)
                proximities.append(distance)

    metrics['building_road_proximities'] = proximities
    metrics['avg_building_road_proximity'] = np.mean(proximities) if proximities else 0
    metrics['median_building_road_proximity'] = np.median(proximities) if proximities else 0

    return metrics


def compute_space_syntax_metrics(G):
    """Compute space syntax metrics."""
    if G.number_of_nodes() < 2:
        return {
            'mean_depth': 0,
            'mean_depth_per_node': {},
            'local_integration': {},
            'choice': {},
            'intelligibility': 0
        }

    G_undir = G.to_undirected()

    if not nx.is_connected(G_undir):
        largest_cc = max(nx.connected_components(G_undir), key=len)
        G_undir = G_undir.subgraph(largest_cc).copy()

    total_depth = 0
    count = 0
    for source in G_undir.nodes():
        lengths = nx.single_source_shortest_path_length(G_undir, source)
        total_depth += sum(lengths.values())
        count += len(lengths)

    mean_depth = total_depth / count if count > 0 else 0

    mean_depth_per_node = {}
    for node in G_undir.nodes():
        lengths = nx.single_source_shortest_path_length(G_undir, node)
        avg = np.mean(list(lengths.values())) if lengths else 0
        mean_depth_per_node[node] = avg

    local_int = {}
    for node in G_undir.nodes():
        lengths = nx.single_source_shortest_path_length(G_undir, node, cutoff=3)
        if len(lengths) > 1:
            total = sum(lengths.values())
            local_int[node] = (len(lengths) - 1) / total if total > 0 else 0
        else:
            local_int[node] = 0

    choice = nx.betweenness_centrality(G_undir, normalized=True)

    degrees = [G_undir.degree(n) for n in local_int.keys()]
    integrations = list(local_int.values())

    if len(degrees) > 1 and np.std(degrees) > 0 and np.std(integrations) > 0:
        corr = np.corrcoef(degrees, integrations)[0, 1]
    else:
        corr = 0

    return {
        'mean_depth': mean_depth,
        'mean_depth_per_node': mean_depth_per_node,
        'local_integration': local_int,
        'choice': choice,
        'intelligibility': corr
    }


print("✓ Metrics functions defined")

In [ ]:
def calculate_bearing(p1, p2):
    """Calculate bearing (0-180°)."""
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    angle = math.atan2(dy, dx)
    bearing = math.degrees(angle)
    # Normalize to 0-180 range
    bearing = bearing % 180
    return bearing


def compute_morphology_metrics(G, pos):
    """Compute morphology metrics from FILTERED graph."""
    metrics = {}
    
    area_km2 = (WINDOW_SIZE_M / 1000.0) ** 2
    metrics['node_density'] = G.number_of_nodes() / area_km2
    
    degrees = [d for _, d in G.degree()]
    metrics['degree_distribution'] = dict(Counter(degrees))
    metrics['avg_degree'] = np.mean(degrees) if degrees else 0
    
    dead_ends = sum(1 for d in degrees if d == 1)
    metrics['dead_end_ratio'] = dead_ends / len(degrees) if degrees else 0
    
    # Segment lengths from FILTERED edges (after removing < 5m)
    lengths = []
    for u, v, key, data in G.edges(keys=True, data=True):
        length = data.get('length', 0)
        lengths.append(length)
    
    metrics['segment_lengths'] = lengths
    metrics['avg_segment_length'] = np.mean(lengths) if lengths else 0
    
    # Orientation from FILTERED edges
    bearings = []
    for u, v in G.edges():
        bearing = calculate_bearing(pos[u], pos[v])
        bearings.append(bearing)
    
    if bearings:
        # Use 18 bins for 10-degree intervals in 0-180 range
        counts, bins = np.histogram(bearings, bins=18, range=(0, 180))
        metrics['orientation_hist'] = (bins, counts)
    else:
        metrics['orientation_hist'] = (np.linspace(0, 180, 19), np.zeros(18))
    
    return metrics


def compute_building_metrics(buildings_gdf, pedestrian_edges_gdf):
    """
    Compute building-related metrics.
    
    Args:
        buildings_gdf: GeoDataFrame of building footprints (normalized coords)
        pedestrian_edges_gdf: GeoDataFrame of pedestrian edges (normalized coords)
    
    Returns:
        Dictionary of building metrics
    """
    from shapely.geometry import MultiLineString
    from shapely.ops import unary_union
    
    metrics = {}
    
    # 1. Footprint area distribution
    areas = []
    perimeters = []
    compactness_values = []
    
    for _, building in buildings_gdf.iterrows():
        if building.geometry is not None and building.geometry.geom_type == 'Polygon':
            area = building.geometry.area
            perimeter = building.geometry.length
            
            areas.append(area)
            perimeters.append(perimeter)
            
            # Compactness: perimeter² / area (higher = more elongated/complex)
            if area > 0:
                compactness = (perimeter ** 2) / area
            else:
                compactness = 0
            compactness_values.append(compactness)
    
    metrics['footprint_areas'] = areas
    metrics['avg_footprint_area'] = np.mean(areas) if areas else 0
    metrics['median_footprint_area'] = np.median(areas) if areas else 0
    
    # 2. Building density (coverage ratio)
    total_building_area = sum(areas)
    window_area = WINDOW_SIZE_M ** 2
    metrics['building_coverage_ratio'] = total_building_area / window_area
    metrics['building_density'] = len(buildings_gdf) / (window_area / 10000)  # per hectare
    
    # 3. Building compactness / shape index
    metrics['compactness_values'] = compactness_values
    metrics['avg_compactness'] = np.mean(compactness_values) if compactness_values else 0
    metrics['median_compactness'] = np.median(compactness_values) if compactness_values else 0
    
    # 4. Building-to-road proximity distribution
    proximities = []
    
    if len(pedestrian_edges_gdf) > 0:
        # Create union of all pedestrian paths for efficient distance calculation
        all_paths = unary_union(pedestrian_edges_gdf.geometry)
        
        for _, building in buildings_gdf.iterrows():
            if building.geometry is not None and building.geometry.geom_type == 'Polygon':
                # Distance from building centroid to nearest pedestrian path
                centroid = building.geometry.centroid
                distance = centroid.distance(all_paths)
                proximities.append(distance)
    
    metrics['building_road_proximities'] = proximities
    metrics['avg_building_road_proximity'] = np.mean(proximities) if proximities else 0
    metrics['median_building_road_proximity'] = np.median(proximities) if proximities else 0
    
    return metrics


def compute_space_syntax_metrics(G):
    """Compute space syntax metrics."""
    if G.number_of_nodes() < 2:
        return {
            'mean_depth': 0,
            'mean_depth_per_node': {},
            'local_integration': {},
            'choice': {},
            'intelligibility': 0
        }
    
    # Convert to undirected and use largest component
    G_undir = G.to_undirected()
    
    if not nx.is_connected(G_undir):
        largest_cc = max(nx.connected_components(G_undir), key=len)
        G_undir = G_undir.subgraph(largest_cc).copy()
    
    # Mean depth
    total_depth = 0
    count = 0
    for source in G_undir.nodes():
        lengths = nx.single_source_shortest_path_length(G_undir, source)
        total_depth += sum(lengths.values())
        count += len(lengths)
    
    mean_depth = total_depth / count if count > 0 else 0
    
    # Mean depth per node
    mean_depth_per_node = {}
    for node in G_undir.nodes():
        lengths = nx.single_source_shortest_path_length(G_undir, node)
        avg = np.mean(list(lengths.values())) if lengths else 0
        mean_depth_per_node[node] = avg
    
    # Local integration (R=3)
    local_int = {}
    for node in G_undir.nodes():
        lengths = nx.single_source_shortest_path_length(G_undir, node, cutoff=3)
        if len(lengths) > 1:
            total = sum(lengths.values())
            local_int[node] = (len(lengths) - 1) / total if total > 0 else 0
        else:
            local_int[node] = 0
    
    # Choice (betweenness)
    choice = nx.betweenness_centrality(G_undir, normalized=True)
    
    # Intelligibility
    degrees = [G_undir.degree(n) for n in local_int.keys()]
    integrations = list(local_int.values())
    
    if len(degrees) > 1 and np.std(degrees) > 0 and np.std(integrations) > 0:
        corr = np.corrcoef(degrees, integrations)[0, 1]
    else:
        corr = 0
    
    return {
        'mean_depth': mean_depth,
        'mean_depth_per_node': mean_depth_per_node,
        'local_integration': local_int,
        'choice': choice,
        'intelligibility': corr
    }


print("✓ Metrics functions defined")

## Load All Cities from GeoJSON

In [ ]:
city_data = {}

print("Loading street networks from GeoJSON files...\n")
print("="*70)

for city_key, city_info in CITIES.items():
    print(f"\n{city_info['name']}:")
    try:
        # Load street network
        G = load_network_from_geojson(city_key)
        
        # Clean and filter
        G_clean, pos = clean_and_filter_graph(G, min_length=MIN_SEGMENT_LENGTH)
        
        # Calculate transformation parameters
        coords = np.array(list(pos.values()))
        min_x, min_y = coords.min(axis=0)
        max_x, max_y = coords.max(axis=0)
        center_x = (min_x + max_x) / 2
        center_y = (min_y + max_y) / 2
        offset_x = WINDOW_SIZE_M / 2 - center_x
        offset_y = WINDOW_SIZE_M / 2 - center_y
        
        # Normalize node positions
        pos_norm = normalize_to_window(pos, WINDOW_SIZE_M)
        
        # Transform edge geometries
        for u, v, key, data in G_clean.edges(keys=True, data=True):
            if 'geometry' in data:
                geom = data['geometry']
                data['geometry_norm'] = transform_geometry(geom, offset_x, offset_y)
        
        # Load buildings and parcels
        buildings_path = Path('inv_city/outputs/geojson') / f"{city_key}_buildings.geojson"
        parcels_path = Path('inv_city/outputs/geojson') / f"{city_key}_parcels.geojson"
        
        buildings_gdf = gpd.read_file(buildings_path)
        parcels_gdf = gpd.read_file(parcels_path)
        
        # Load pedestrian edges for building metrics (before transform)
        edges_path = Path('inv_city/outputs/geojson') / f"{city_key}_edges.geojson"
        all_edges = gpd.read_file(edges_path)
        
        # Filter for pedestrian paths
        pedestrian_types = ['footway', 'path', 'pedestrian', 'steps']
        def is_pedestrian(hw):
            if hw is None: return False
            if hasattr(hw, '__iter__') and not isinstance(hw, str):
                hw = hw[0] if len(hw) > 0 else None
            return hw in pedestrian_types
        
        pedestrian_edges = all_edges[all_edges['highway'].apply(is_pedestrian)].copy()
        
        # Transform buildings, parcels, and pedestrian edges to normalized coordinates
        from shapely.affinity import translate
        buildings_gdf['geometry'] = buildings_gdf['geometry'].apply(
            lambda geom: translate(geom, xoff=offset_x, yoff=offset_y)
        )
        parcels_gdf['geometry'] = parcels_gdf['geometry'].apply(
            lambda geom: translate(geom, xoff=offset_x, yoff=offset_y)
        )
        pedestrian_edges['geometry'] = pedestrian_edges['geometry'].apply(
            lambda geom: translate(geom, xoff=offset_x, yoff=offset_y) if geom is not None else None
        )
        
        print(f"  Loaded {len(buildings_gdf)} buildings, {len(parcels_gdf)} parcels")
        
        # Compute metrics
        morph = compute_morphology_metrics(G_clean, pos_norm)
        syntax = compute_space_syntax_metrics(G_clean)
        building_metrics = compute_building_metrics(buildings_gdf, pedestrian_edges)
        
        city_data[city_key] = {
            'graph': G_clean,
            'pos': pos_norm,
            'morphology': morph,
            'syntax': syntax,
            'building_metrics': building_metrics,
            'buildings': buildings_gdf,
            'parcels': parcels_gdf,
            'pedestrian_edges': pedestrian_edges,
            'transform': {'offset_x': offset_x, 'offset_y': offset_y}
        }
        
        print(f"  ✓ Success!\n")
        
    except Exception as e:
        print(f"  ✗ Error: {e}\n")
        import traceback
        traceback.print_exc()

print("="*70)
print(f"\n✓ Loaded {len(city_data)} cities successfully\n")

## A1. Visualize All 4 Networks

## A0. Base Maps (Buildings + Parcels)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))axes = axes.flatten()for idx, (city_key, data) in enumerate(city_data.items()):    ax = axes[idx]    buildings = data['buildings']    pedestrian_edges = data['pedestrian_edges']    ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,                           fill=False, edgecolor='black', linestyle='-', linewidth=2))    for _, row in buildings.iterrows():        if row.geometry is not None and row.geometry.geom_type == 'Polygon':            xs, ys = row.geometry.exterior.xy            ax.fill(xs, ys, color='darkgray', alpha=0.7, edgecolor='black', linewidth=0.5, zorder=1)    color = CITIES[city_key]['color']    for _, edge in pedestrian_edges.iterrows():        if edge.geometry is not None:            xs, ys = edge.geometry.xy            ax.plot(xs, ys, color=color, linewidth=2, alpha=0.8, zorder=2)    ax.set_xlim(-20, WINDOW_SIZE_M + 20)    ax.set_ylim(-20, WINDOW_SIZE_M + 20)    ax.set_aspect('equal')    ax.set_title(        f"{CITIES[city_key]['name']}\n{len(buildings)} buildings, {len(pedestrian_edges)} paths",        fontsize=12, fontweight='bold'    )    ax.set_xlabel('X (meters)')    ax.set_ylabel('Y (meters)')    ax.grid(True, alpha=0.2)legend_elements = [    Patch(facecolor='darkgray', edgecolor='black', label='Buildings', alpha=0.7),    plt.Line2D([0], [0], color='gray', linewidth=2, label='Pedestrian Paths')]fig.legend(handles=legend_elements, loc='upper right', fontsize=11)plt.suptitle('Base Maps: Buildings + Pedestrian Networks',             fontsize=16, fontweight='bold', y=0.995)plt.tight_layout()plt.savefig('outputs/visualizations/A0_base_maps.svg', format='svg', bbox_inches='tight', dpi=300)print("Saved: outputs/visualizations/A0_base_maps.svg")plt.show()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes = axes.flatten()

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    G = data['graph']
    pos = data['pos']
    color = CITIES[city_key]['color']
    
    # Window boundary
    ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,
                           fill=False, edgecolor='gray', linestyle='--', linewidth=1.5))
    
    # Draw edges using actual geometry
    for u, v, key, edata in G.edges(keys=True, data=True):
        if 'geometry_norm' in edata:
            geom = edata['geometry_norm']
            xs, ys = geom.xy
            ax.plot(xs, ys, color=color, linewidth=1.5, alpha=0.7, zorder=1)
        else:
            # Fallback to straight line
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, color=color, linewidth=1.5, alpha=0.7, zorder=1)
    
    # Draw nodes colored by degree
    degrees = dict(G.degree())
    max_degree = max(degrees.values()) if degrees else 1
    
    for node in G.nodes():
        degree = degrees[node]
        color_val = degree / max_degree
        node_color = plt.cm.RdYlBu_r(color_val)
        ax.scatter(pos[node][0], pos[node][1], s=40, c=[node_color],
                  zorder=2, edgecolors='black', linewidths=0.5)
    
    ax.set_xlim(-20, WINDOW_SIZE_M + 20)
    ax.set_ylim(-20, WINDOW_SIZE_M + 20)
    ax.set_aspect('equal')
    ax.set_title(
        f"{CITIES[city_key]['name']}\n{G.number_of_nodes()} nodes, {G.number_of_edges()} edges",
        fontsize=12, fontweight='bold'
    )
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.grid(True, alpha=0.3)

# Add colorbar legend
import matplotlib.cm as cm
import matplotlib.colors as mcolors

norm = mcolors.Normalize(vmin=0, vmax=1)
sm = cm.ScalarMappable(cmap=plt.cm.RdYlBu_r, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=axes, orientation='horizontal',
                     pad=0.05, fraction=0.05, aspect=40)
cbar.set_label('Node Degree (normalized)', fontsize=11)

plt.suptitle('Reference Street Networks (actual geometry)',
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()

# Save as SVG
plt.savefig('outputs/visualizations/A1_street_networks.svg', format='svg', bbox_inches='tight', dpi=300)
print("Saved: outputs/visualizations/A1_street_networks.svg")

plt.show()


## A2. Summary Statistics

In [ ]:
summary_rows = []

for city_key, data in city_data.items():
    G = data['graph']
    morph = data['morphology']
    syntax = data['syntax']
    
    summary_rows.append({
        'City': CITIES[city_key]['name'],
        'Nodes': G.number_of_nodes(),
        'Edges': G.number_of_edges(),
        'Density (n/km²)': f"{morph['node_density']:.1f}",
        'Avg Degree': f"{morph['avg_degree']:.2f}",
        'Dead-End Ratio': f"{morph['dead_end_ratio']:.3f}",
        'Avg Seg Length (m)': f"{morph['avg_segment_length']:.1f}",
        'Mean Depth': f"{syntax['mean_depth']:.2f}",
        'Intelligibility': f"{syntax['intelligibility']:.3f}"
    })

df_summary = pd.DataFrame(summary_rows)
print("\n" + "="*100)
print(" "*35 + "REFERENCE CITIES SUMMARY")
print("="*100)
print(df_summary.to_string(index=False))
print("="*100)

## A3. Degree Distribution

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))axes = axes.flatten()for idx, (city_key, data) in enumerate(city_data.items()):    ax = axes[idx]    degree_dist = data['morphology']['degree_distribution']        degrees = sorted(degree_dist.keys())    counts = [degree_dist[d] for d in degrees]    total = sum(counts)    probs = [c / total for c in counts]        color = CITIES[city_key]['color']    ax.bar(degrees, probs, color=color, alpha=0.7, edgecolor='black', linewidth=1)    ax.set_xlabel('Node Degree', fontsize=11)    ax.set_ylabel('Probability', fontsize=11)    ax.set_title(CITIES[city_key]['name'], fontweight='bold', fontsize=12)    ax.grid(True, alpha=0.3, axis='y')    ax.set_xticks(degrees)plt.suptitle('Degree Distributions', fontsize=14, fontweight='bold')plt.tight_layout()# Save as SVGplt.savefig('outputs/visualizations/A3_degree_distributions.svg', format='svg', bbox_inches='tight', dpi=300)print('Saved: outputs/visualizations/A3_degree_distributions.svg')plt.show()

## A4. Orientation Rose Diagrams

In [ ]:
fig = plt.figure(figsize=(14, 10))

for idx, (city_key, data) in enumerate(city_data.items(), 1):
    ax = fig.add_subplot(2, 2, idx, projection='polar')
    
    bins, counts = data['morphology']['orientation_hist']
    bin_centers = (bins[:-1] + bins[1:]) / 2
    
    # Show only 0-180° range (actual street orientation)
    theta = np.deg2rad(bin_centers)
    
    # Normalize
    total = sum(counts)
    probs = counts / total if total > 0 else counts
    
    color = CITIES[city_key]['color']
    width = np.deg2rad(bins[1] - bins[0])
    
    ax.bar(theta, probs, width=width, color=color, alpha=0.7,
           edgecolor='black', linewidth=0.5)
    
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    ax.set_title(CITIES[city_key]['name'], fontweight='bold', fontsize=12, pad=20)
    ax.set_ylim(0, max(probs) * 1.1 if max(probs) > 0 else 0.1)
    ax.set_thetamax(180)  # Limit to 180 degrees

plt.suptitle('Street Orientation Diagrams (0-180°)', fontsize=14, fontweight='bold', y=0.98)
plt.tight_layout()

# Save as SVG
plt.savefig('outputs/visualizations/A4_orientation_rose.svg', format='svg', bbox_inches='tight', dpi=300)
print("Saved: outputs/visualizations/A4_orientation_rose.svg")

plt.show()


## A5. Local Integration Maps

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))axes = axes.flatten()for idx, (city_key, data) in enumerate(city_data.items()):    ax = axes[idx]    G = data['graph']    pos = data['pos']    local_int = data['syntax']['local_integration']        # Window    ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,                           fill=False, edgecolor='gray', linestyle='--', linewidth=1))        # Draw edges (gray)    for u, v in G.edges():        x = [pos[u][0], pos[v][0]]        y = [pos[u][1], pos[v][1]]        ax.plot(x, y, color='lightgray', linewidth=1, alpha=0.5, zorder=1)        # Draw nodes colored by local integration    values = list(local_int.values())    if values:        vmin, vmax = min(values), max(values)                for node in G.nodes():            if node in local_int:                val = local_int[node]                norm_val = (val - vmin) / (vmax - vmin + 1e-10)                color = plt.cm.hot(norm_val)                                ax.scatter(pos[node][0], pos[node][1], s=60, c=[color],                          zorder=2, edgecolors='black', linewidths=0.5)        ax.set_xlim(-20, WINDOW_SIZE_M + 20)    ax.set_ylim(-20, WINDOW_SIZE_M + 20)    ax.set_aspect('equal')    ax.set_title(f"{CITIES[city_key]['name']}\nLocal Integration (R=3)",                fontweight='bold', fontsize=12)    ax.set_xlabel('X (meters)')    ax.set_ylabel('Y (meters)')    ax.grid(True, alpha=0.3)plt.suptitle('Local Integration Maps (warm = high integration)',            fontsize=16, fontweight='bold', y=0.995)plt.tight_layout()# Save as SVGplt.savefig('outputs/visualizations/A5_local_integration.svg', format='svg', bbox_inches='tight', dpi=300)print('Saved: outputs/visualizations/A5_local_integration.svg')plt.show()

## A6. Choice (Betweenness) Maps

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))axes = axes.flatten()for idx, (city_key, data) in enumerate(city_data.items()):    ax = axes[idx]    G = data['graph']    pos = data['pos']    choice = data['syntax']['choice']        ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,                           fill=False, edgecolor='gray', linestyle='--', linewidth=1))        # Edges    for u, v in G.edges():        ax.plot([pos[u][0], pos[v][0]], [pos[u][1], pos[v][1]],               color='lightgray', linewidth=1, alpha=0.5, zorder=1)        # Nodes colored by choice    values = list(choice.values())    if values:        vmin, vmax = min(values), max(values)                for node in G.nodes():            if node in choice:                val = choice[node]                norm_val = (val - vmin) / (vmax - vmin + 1e-10)                color_val = plt.cm.viridis(norm_val)                                ax.scatter(pos[node][0], pos[node][1], s=60, c=[color_val],                          zorder=2, edgecolors='black', linewidths=0.5)        ax.set_xlim(-20, WINDOW_SIZE_M + 20)    ax.set_ylim(-20, WINDOW_SIZE_M + 20)    ax.set_aspect('equal')    ax.set_title(f"{CITIES[city_key]['name']}\nChoice (Betweenness)",                fontweight='bold', fontsize=12)    ax.set_xlabel('X (meters)')    ax.set_ylabel('Y (meters)')    ax.grid(True, alpha=0.3)plt.suptitle('Choice Maps (through-movement corridors)',            fontsize=16, fontweight='bold', y=0.995)plt.tight_layout()# Save as SVGplt.savefig('outputs/visualizations/A6_choice_betweenness.svg', format='svg', bbox_inches='tight', dpi=300)print('Saved: outputs/visualizations/A6_choice_betweenness.svg')plt.show()

## A7. Mean Depth Maps

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))axes = axes.flatten()for idx, (city_key, data) in enumerate(city_data.items()):    ax = axes[idx]    G = data['graph']    pos = data['pos']    mean_depth_nodes = data['syntax']['mean_depth_per_node']        ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,                           fill=False, edgecolor='gray', linestyle='--', linewidth=1))        # Edges    for u, v in G.edges():        ax.plot([pos[u][0], pos[v][0]], [pos[u][1], pos[v][1]],               color='lightgray', linewidth=1, alpha=0.5, zorder=1)        # Nodes colored by mean depth    values = list(mean_depth_nodes.values())    if values:        vmin, vmax = min(values), max(values)                for node in G.nodes():            if node in mean_depth_nodes:                val = mean_depth_nodes[node]                norm_val = (val - vmin) / (vmax - vmin + 1e-10)                color_val = plt.cm.coolwarm_r(norm_val)                                ax.scatter(pos[node][0], pos[node][1], s=60, c=[color_val],                          zorder=2, edgecolors='black', linewidths=0.5)        ax.set_xlim(-20, WINDOW_SIZE_M + 20)    ax.set_ylim(-20, WINDOW_SIZE_M + 20)    ax.set_aspect('equal')    ax.set_title(f"{CITIES[city_key]['name']}\nMean Depth",                fontweight='bold', fontsize=12)    ax.set_xlabel('X (meters)')    ax.set_ylabel('Y (meters)')    ax.grid(True, alpha=0.3)plt.suptitle('Mean Depth Maps (blue = central, red = peripheral)',            fontsize=16, fontweight='bold', y=0.995)plt.tight_layout()# Save as SVGplt.savefig('outputs/visualizations/A7_mean_depth.svg', format='svg', bbox_inches='tight', dpi=300)print('Saved: outputs/visualizations/A7_mean_depth.svg')plt.show()

## A8. Intelligibility Scatter

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))axes = axes.flatten()for idx, (city_key, data) in enumerate(city_data.items()):    ax = axes[idx]    G = data['graph'].to_undirected()    local_int = data['syntax']['local_integration']    intelligibility = data['syntax']['intelligibility']        degrees = [G.degree(n) for n in local_int.keys()]    integrations = list(local_int.values())        color = CITIES[city_key]['color']    ax.scatter(degrees, integrations, alpha=0.6, s=30, c=color,              edgecolors='black', linewidths=0.5)        # Fit line    if len(degrees) > 1:        z = np.polyfit(degrees, integrations, 1)        p = np.poly1d(z)        x_line = np.linspace(min(degrees), max(degrees), 100)        ax.plot(x_line, p(x_line), 'r--', linewidth=2, alpha=0.7)        ax.set_xlabel('Connectivity (Degree)', fontsize=11)    ax.set_ylabel('Local Integration', fontsize=11)    ax.set_title(f"{CITIES[city_key]['name']}\nr = {intelligibility:.3f}",                fontweight='bold', fontsize=12)    ax.grid(True, alpha=0.3)plt.suptitle('Intelligibility: Degree vs Local Integration',            fontsize=14, fontweight='bold')plt.tight_layout()# Save as SVGplt.savefig('outputs/visualizations/A8_intelligibility.svg', format='svg', bbox_inches='tight', dpi=300)print('Saved: outputs/visualizations/A8_intelligibility.svg')plt.show()

## A9. Cross-City Segment Lengths

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(18, 4))for idx, (city_key, data) in enumerate(city_data.items()):    ax = axes[idx]    lengths = data['morphology']['segment_lengths']        if lengths:        counts, bins = np.histogram(lengths, bins=20, range=(MIN_SEGMENT_LENGTH, 120))        bin_centers = (bins[:-1] + bins[1:]) / 2        total = sum(counts)        probs = counts / total if total > 0 else counts                color = CITIES[city_key]['color']        ax.bar(bin_centers, probs, width=(bins[1]-bins[0])*0.9,              color=color, alpha=0.7, edgecolor='black', linewidth=0.5)        ax.set_xlabel('Length (m)', fontsize=10)    ax.set_ylabel('Probability' if idx == 0 else '', fontsize=10)    ax.set_title(CITIES[city_key]['name'], fontweight='bold', fontsize=11)    ax.set_ylim(0, 0.2)    ax.grid(True, alpha=0.3, axis='y')plt.suptitle('Segment Length Distributions (Aligned)',            fontsize=13, fontweight='bold')plt.tight_layout()# Save as SVGplt.savefig('outputs/visualizations/A9_segment_lengths.svg', format='svg', bbox_inches='tight', dpi=300)print('Saved: outputs/visualizations/A9_segment_lengths.svg')plt.show()

## A13. Node Degree Maps

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))axes = axes.flatten()degree_colors = {1: 'red', 2: 'orange', 3: 'yellow', 4: 'green', 5: 'blue', 6: 'purple'}for idx, (city_key, data) in enumerate(city_data.items()):    ax = axes[idx]    G = data['graph']    pos = data['pos']        ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,                           fill=False, edgecolor='gray', linestyle='--', linewidth=1))        # Edges    for u, v in G.edges():        ax.plot([pos[u][0], pos[v][0]], [pos[u][1], pos[v][1]],               color='lightgray', linewidth=1, alpha=0.4, zorder=1)        # Nodes sized and colored by degree    degrees = dict(G.degree())        for node in G.nodes():        degree = degrees[node]        size = 30 + degree * 20        color = degree_colors.get(degree, 'gray')                ax.scatter(pos[node][0], pos[node][1], s=size, c=color,                  zorder=2, edgecolors='black', linewidths=0.5, alpha=0.8)        ax.set_xlim(-20, WINDOW_SIZE_M + 20)    ax.set_ylim(-20, WINDOW_SIZE_M + 20)    ax.set_aspect('equal')    ax.set_title(f"{CITIES[city_key]['name']}\nNode Degree Map",                fontweight='bold', fontsize=12)    ax.set_xlabel('X (meters)')    ax.set_ylabel('Y (meters)')    ax.grid(True, alpha=0.3)plt.suptitle('Node Degree Maps (size & color by degree)',            fontsize=16, fontweight='bold', y=0.995)plt.tight_layout()# Save as SVGplt.savefig('outputs/visualizations/A13_node_degree.svg', format='svg', bbox_inches='tight', dpi=300)print('Saved: outputs/visualizations/A13_node_degree.svg')plt.show()

## A14. Core + Corridor Overlap

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))axes = axes.flatten()TOP_PERCENT = 0.2for idx, (city_key, data) in enumerate(city_data.items()):    ax = axes[idx]    G = data['graph']    pos = data['pos']    local_int = data['syntax']['local_integration']    choice = data['syntax']['choice']        ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,                           fill=False, edgecolor='gray', linestyle='--', linewidth=1))        # Find top 20%    int_values = sorted(local_int.values(), reverse=True)    choice_values = sorted(choice.values(), reverse=True)        int_threshold = int_values[int(len(int_values) * TOP_PERCENT)] if int_values else 0    choice_threshold = choice_values[int(len(choice_values) * TOP_PERCENT)] if choice_values else 0        high_int = {n for n, v in local_int.items() if v >= int_threshold}    high_choice = {n for n, v in choice.items() if v >= choice_threshold}    overlap = high_int & high_choice        # Edges    for u, v in G.edges():        ax.plot([pos[u][0], pos[v][0]], [pos[u][1], pos[v][1]],               color='lightgray', linewidth=1, alpha=0.3, zorder=1)        # All nodes (small gray)    for node in G.nodes():        ax.scatter(pos[node][0], pos[node][1], s=10, c='lightgray',                  zorder=2, alpha=0.5)        # High integration only (blue)    for node in high_int - overlap:        ax.scatter(pos[node][0], pos[node][1], s=60, c='blue',                  zorder=3, edgecolors='black', linewidths=0.5, alpha=0.7)        # High choice only (green)    for node in high_choice - overlap:        ax.scatter(pos[node][0], pos[node][1], s=60, c='green',                  zorder=3, edgecolors='black', linewidths=0.5, alpha=0.7)        # Overlap (red)    for node in overlap:        ax.scatter(pos[node][0], pos[node][1], s=80, c='red',                  zorder=4, edgecolors='black', linewidths=1, alpha=0.9)        ax.set_xlim(-20, WINDOW_SIZE_M + 20)    ax.set_ylim(-20, WINDOW_SIZE_M + 20)    ax.set_aspect('equal')    ax.set_title(f"{CITIES[city_key]['name']}\nCore+Corridor ({len(overlap)} overlap)",                fontweight='bold', fontsize=12)    ax.set_xlabel('X (meters)')    ax.set_ylabel('Y (meters)')    ax.grid(True, alpha=0.3)legend_elements = [    Patch(facecolor='blue', edgecolor='black', label='High Integration'),    Patch(facecolor='green', edgecolor='black', label='High Choice'),    Patch(facecolor='red', edgecolor='black', label='Overlap (Both)')]fig.legend(handles=legend_elements, loc='upper center', ncol=3,          bbox_to_anchor=(0.5, 0.99))plt.suptitle('Core + Corridor Overlap (Top 20%)',            fontsize=16, fontweight='bold', y=0.975)plt.tight_layout()# Save as SVGplt.savefig('outputs/visualizations/A14_core_corridor.svg', format='svg', bbox_inches='tight', dpi=300)print('Saved: outputs/visualizations/A14_core_corridor.svg')plt.show()

## Save Reference Data

## A15. Isovist-Based Visibility Analysis

In [ ]:
def compute_isovist(observer_point, buildings_gdf, max_radius=100, num_rays=360):
    """
    Compute isovist (viewshed) from an observer point.
    
    Args:
        observer_point: (x, y) tuple
        buildings_gdf: GeoDataFrame of building polygons
        max_radius: Maximum visibility distance
        num_rays: Number of rays to cast (angular resolution)
    
    Returns:
        Polygon representing visible area
    """
    from shapely.geometry import Point, LineString, Polygon
    from shapely.ops import unary_union
    import numpy as np
    
    observer = Point(observer_point)
    visible_points = []
    
    # Cast rays in all directions
    for i in range(num_rays):
        angle = 2 * np.pi * i / num_rays
        dx = max_radius * np.cos(angle)
        dy = max_radius * np.sin(angle)
        
        ray_end = (observer_point[0] + dx, observer_point[1] + dy)
        ray = LineString([observer_point, ray_end])
        
        # Find closest intersection with buildings
        min_dist = max_radius
        closest_point = ray_end
        
        for _, building in buildings_gdf.iterrows():
            if building.geometry is None:
                continue
            
            if ray.intersects(building.geometry):
                intersection = ray.intersection(building.geometry)
                
                if intersection.is_empty:
                    continue
                
                # Handle different intersection types
                if intersection.geom_type == 'Point':
                    dist = observer.distance(intersection)
                    if dist < min_dist:
                        min_dist = dist
                        closest_point = (intersection.x, intersection.y)
                elif intersection.geom_type == 'MultiPoint':
                    for pt in intersection.geoms:
                        dist = observer.distance(pt)
                        if dist < min_dist:
                            min_dist = dist
                            closest_point = (pt.x, pt.y)
                elif intersection.geom_type == 'LineString':
                    for coord in intersection.coords:
                        pt = Point(coord)
                        dist = observer.distance(pt)
                        if dist < min_dist:
                            min_dist = dist
                            closest_point = coord
        
        visible_points.append(closest_point)
    
    # Create isovist polygon
    if len(visible_points) > 2:
        isovist = Polygon(visible_points)
        return isovist
    else:
        return None


def compute_isovist_metrics(isovist_polygon):
    """Compute metrics from isovist polygon."""
    if isovist_polygon is None or isovist_polygon.is_empty:
        return {'area': 0, 'perimeter': 0, 'compactness': 0}
    
    area = isovist_polygon.area
    perimeter = isovist_polygon.length
    
    # Compactness: 4π * area / perimeter²
    compactness = 4 * np.pi * area / (perimeter ** 2) if perimeter > 0 else 0
    
    return {
        'area': area,
        'perimeter': perimeter,
        'compactness': compactness
    }


print("✓ Isovist functions defined")


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes = axes.flatten()

GRID_SPACING = 10  # 10m × 10m grid

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    buildings = data['buildings']
    pos = data['pos']
    G = data['graph']
    
    # Window boundary
    ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,
                           fill=False, edgecolor='black', linestyle='-', linewidth=2))
    
    # Draw buildings
    for _, row in buildings.iterrows():
        if row.geometry is not None and row.geometry.geom_type == 'Polygon':
            xs, ys = row.geometry.exterior.xy
            ax.fill(xs, ys, color='darkgray', alpha=0.7, edgecolor='black', linewidth=0.5, zorder=1)
    
    # Create grid of observer points (every 10m)
    grid_x = np.arange(GRID_SPACING, WINDOW_SIZE_M, GRID_SPACING)
    grid_y = np.arange(GRID_SPACING, WINDOW_SIZE_M, GRID_SPACING)
    
    print(f"Computing {len(grid_x) * len(grid_y)} isovists for {CITIES[city_key]['name']}...")
    
    # Store all isovist polygons
    all_isovists = []
    all_areas = []
    
    # Compute isovist for each grid point
    for x in grid_x:
        for y in grid_y:
            observer_point = (x, y)
            isovist = compute_isovist(observer_point, buildings, max_radius=50, num_rays=72)
            
            if isovist is not None and not isovist.is_empty:
                all_isovists.append(isovist)
                metrics = compute_isovist_metrics(isovist)
                all_areas.append(metrics['area'])
    
    print(f"  Computed {len(all_isovists)} valid isovists")
    
    # Visualize isovists with color based on area
    if all_areas:
        min_area = min(all_areas)
        max_area = max(all_areas)
        
        for isovist, area in zip(all_isovists, all_areas):
            # Color by area (normalized)
            norm_area = (area - min_area) / (max_area - min_area + 1e-10)
            color = plt.cm.YlOrRd(norm_area)
            
            xs, ys = isovist.exterior.xy
            ax.fill(xs, ys, color=color, alpha=0.3, edgecolor='none', zorder=2)
        
        # Draw grid points
        for x in grid_x:
            for y in grid_y:
                ax.plot(x, y, 'k.', markersize=2, alpha=0.5, zorder=3)
        
        avg_area = np.mean(all_areas)
        title_text = f"{CITIES[city_key]['name']}\n{len(all_isovists)} isovists, Avg Area: {avg_area:.0f} m²"
    else:
        title_text = f"{CITIES[city_key]['name']}\nNo isovists computed"
    
    ax.set_xlim(-20, WINDOW_SIZE_M + 20)
    ax.set_ylim(-20, WINDOW_SIZE_M + 20)
    ax.set_aspect('equal')
    ax.set_title(title_text, fontsize=12, fontweight='bold')
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.grid(True, alpha=0.2)

# Add legend and colorbar
from matplotlib.patches import Patch
import matplotlib.cm as cm
import matplotlib.colors as mcolors

legend_elements = [
    Patch(facecolor='darkgray', edgecolor='black', label='Buildings', alpha=0.7),
    plt.Line2D([0], [0], marker='.', color='w', markerfacecolor='k', markersize=6, label='Observer Grid (10m)')
]
fig.legend(handles=legend_elements, loc='upper left', fontsize=11)

# Add colorbar for visibility area
norm = mcolors.Normalize(vmin=0, vmax=1)
sm = cm.ScalarMappable(cmap=plt.cm.YlOrRd, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=axes, orientation='horizontal',
                     pad=0.05, fraction=0.05, aspect=40)
cbar.set_label('Visible Area (normalized: yellow=low, red=high)', fontsize=11)

plt.suptitle('Isovist Analysis: Visibility from 10m×10m Grid',
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()

# Save as SVG
plt.savefig('outputs/visualizations/A15_isovist_analysis.svg', format='svg', bbox_inches='tight', dpi=300)
print("Saved: outputs/visualizations/A15_isovist_analysis.svg")

plt.show()


In [ ]:
with open('outputs/data/reference_cities_data.pkl', 'wb') as f:    pickle.dump(city_data, f)print("✓ Reference data saved to: outputs/data/reference_cities_data.pkl")print(f"\nCities saved: {list(city_data.keys())}")print("\n" + "="*70)print("METRICS SUMMARY")print("="*70)for city_key, data in city_data.items():    print(f"\n{CITIES[city_key]['name']}:")    print(f"  Network: {data['graph'].number_of_nodes()} nodes, {data['graph'].number_of_edges()} edges")    print(f"  Buildings: {len(data['buildings'])} total")    print(f"  - Avg area: {data['building_metrics']['avg_footprint_area']:.1f} m²")    print(f"  - Coverage: {data['building_metrics']['building_coverage_ratio']:.3f}")    print(f"  - Avg compactness: {data['building_metrics']['avg_compactness']:.1f}")    print(f"  - Avg dist to path: {data['building_metrics']['avg_building_road_proximity']:.1f} m")print("\n" + "="*70)print("Ready for Step 2: Network Generation")